# Stars interacting gravitationally in VPython

This is essentially the demo code written by Bruce Sherwood (https://github.com/BruceSherwood/vpython-jupyter/blob/master/Demos/Stars.ipynb). 

The original was a block of Python pasted into a single Jupyter cell. This variant just breaks things up more, moving code out to functions and minimizing the length of the (most important) cells that create a scene and run the simulation.

Note that vpython is the only import here. Attempts to combine this sort of animation with my own Numpy code (as with Bokeh in Animations.ipynb) have so far proved frustrating. Maybe the next attempt will crack it...

In [1]:
from vpython import *

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Some global variables:

In [2]:
Nstars = 20  # change this to have more or fewer stars
Stars = []
L = 4e10
G = 6.7e-11 # Universal gravitational constant

# Typical values
Msun = 2E30
Rsun = 2E9
vsun = 0.8*sqrt(G*Msun/Rsun)

Set basic properties for the scene:

In [3]:
def makeScene(scene):
    scene.width = scene.height = 600

    # Display text below the 3D graphics:
    scene.title = "Stars interacting gravitationally"

    scene.caption = """Right button drag or Ctrl-drag to rotate "camera" to view scene.
    To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
      On a two-button mouse, middle is left + right.
    Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""

    scene.range = 2*L
    scene.forward = vec(-1,-1,-1)

    makeAxes()

def makeAxes():
    xaxis = curve(color=color.gray(0.5), radius=3e8)
    xaxis.append(vec(0,0,0))
    xaxis.append(vec(L,0,0))
    yaxis = curve(color=color.gray(0.5), radius=3e8)
    yaxis.append(vec(0,0,0))
    yaxis.append(vec(0,L,0))
    zaxis = curve(color=color.gray(0.5), radius=3e8)
    zaxis.append(vec(0,0,0))
    zaxis.append(vec(0,0,L))

Add some moving spheres. Note that these are complex objects which store all their own properties: mass, momentum, etc, as well as graphical features like size and color.

In [4]:
def makeStars():
    star_colors = [color.red, color.green, color.blue,
                  color.yellow, color.cyan, color.magenta]

    psum = vec(0,0,0)
    for i in range(Nstars):
        star = sphere(pos=L*vec.random(), make_trail=True, retain=150, trail_radius=3e8)
        R = Rsun/2+Rsun*random()
        star.radius = R
        star.mass = Msun*(R/Rsun)**3
        star.momentum = vec.random()*vsun*star.mass
        star.color = star.trail_color = star_colors[i % 6]
        Stars.append( star )
        psum = psum + star.momentum

    #make total initial momentum equal zero
    for i in range(Nstars):
        Stars[i].momentum = Stars[i].momentum - psum/Nstars

A very simple inegrator:

In [5]:
def computeForces():
    global hitlist, Stars
    hitlist = []
    N = len(Stars)
    for i in range(N):
        si = Stars[i]
        if si is None: continue
        F = vec(0,0,0)
        pos1 = si.pos
        m1 = si.mass
        radius = si.radius
        for j in range(N):
            if i == j: continue
            sj = Stars[j]
            if sj is None: continue
            r = sj.pos - pos1
            rmag2 = mag2(r)
            if rmag2 <= (radius+sj.radius)**2: hitlist.append([i,j])
            F = F + (G*m1*sj.mass/(rmag2**1.5))*r
        si.momentum = si.momentum + F*dt
    return hitlist

Collisions are totally inelastic. One star increases in mass/momentum, the other is removed.

In [6]:
def processHits(hitlist):
        # If any collisions took place, merge those stars
        hit = len(hitlist)-1
        while hit > 0:
            s1 = Stars[hitlist[hit][0]]
            s2 = Stars[hitlist[hit][1]]
            if (s1 is None) or (s2 is None): continue

            mass = s1.mass + s2.mass
            momentum = s1.momentum + s2.momentum
            pos = (s1.mass*s1.pos + s2.mass*s2.pos) / mass
            s1.color = s1.trail_color = (s1.mass*s1.color + s2.mass*s2.color) / mass
            R = Rsun*(mass / Msun)**(1/3)

            s1.clear_trail()
            s2.clear_trail()
            s2.visible = False

            s1.mass = mass
            s1.momentum = momentum
            s1.pos = pos
            s1.radius = R
            Stars[hitlist[hit][1]] = None
            hit -= 1

The next cell creates everything and displays the static initial scene. If the graphic fails to show (which is not unusual), try Kernel > Restart & Clear Output

In [7]:
scene = canvas() # This is needed in Jupyter notebook and lab to make programs easily rerunnable
makeScene(scene)
makeStars()

<IPython.core.display.Javascript object>

Now run the animation. A kernel interrupt will stop the graphic, but stopping all computation may need a restart.

In [8]:
dt = 1000
hitlist = []
try:
    while True:
        rate(100)
        hitlist = computeForces()

        # Having updated all momenta, now update all positions
        for star in Stars:
            if star is None: continue
            star.pos = star.pos + star.momentum*(dt/star.mass)
            
        # deal with collisions (treated as inelastic mergers)
        processHits(hitlist)
        
except KeyboardInterrupt:
    pass # just stop with no traceback        